In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
df = pd.read_excel("covid_UPD.xlsx")

In [69]:
df.shape

(76219, 22)

проверим на пропущенные значения

In [70]:
df.head()

,week,region,oced,unemployed,cv_gender,cv_birthday_2000,cv_birthday_1990,cv_birthday_1980,cv_birthday_1970,cv_birthday_1960,...,women,infections,deaths,healed,tests,Rt,level,yandex_continuous,yandex_level,pass
0,2020-04-06,Адыгея,"Строительство, ремонт, стройматериалы, недвижи...",19,18,0,3,12,19,19,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
1,2020-04-06,Адыгея,Машиностроение,2,1,0,0,1,1,2,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
2,2020-04-06,Адыгея,Юриспруденция,13,6,0,2,11,13,13,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
3,2020-04-06,Адыгея,Пищевая промышленность,9,2,1,2,6,9,9,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
4,2020-04-06,Адыгея,"Государственная служба, некоммерческие организ...",17,8,0,3,14,17,17,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0


### из-за низкой репрезентативности удалим регионы с суммарным кол-вом поданных заявок меньше 1000

In [71]:
count_sum_unemployed_by_regions = df.groupby("region").agg(sum_uneployed=("unemployed", "sum")).sort_values(by="sum_uneployed")
count_sum_unemployed_by_regions

,sum_uneployed
region,
Байконур,63
Чукотский АО,144
Ненецкий АО,820
Магаданская,1369
Сахалинская,4892
...,...
Ростовская,149789
Московская,191943
Краснодарский край,224576


In [72]:
regions_to_del = count_sum_unemployed_by_regions[count_sum_unemployed_by_regions.sum_uneployed < 1000].index
df = df.drop(df[df.region.isin(regions_to_del)].index, axis=0)
df.shape

(75827, 22)

### также удалим сильно ывделяющиеся сферы трудовой деятельности с низким кол-вом сэмплов

In [73]:
count_sum_unemployed_by_prof = df.groupby("oced").agg(oced_count = ("oced", "count")).sort_values(by="oced_count")
count_sum_unemployed_by_prof

,oced_count
oced,
Логистика,56
Начало трудовой деятельности,167
"Консалтинг, стратегическое развитие, управление",1443
"Химическая, нефтехимическая, топливная промышленность",1785
"Лесная, деревообрабатывающая, целлюлозно-бумажная промышленность",1935
"Металлургия, металлообработка",1954
Добывающая промышленность,2089
Домашний персонал,2115
Электроэнергетика,2168


In [74]:
prof_to_del = count_sum_unemployed_by_prof[count_sum_unemployed_by_prof.oced_count < 1000].index
df = df.drop(df[df.oced.isin(prof_to_del)].index, axis=0)
df.shape

(75604, 22)

### также посмотрим на колво заражений, смертей и выздоравлений в разных регионах

In [104]:
covid_df = df.groupby("region").agg(sum_infections = ("infections", "sum"), population=("population", "mean"), sum_healed=("healed", "sum"), sum_deathed=("deaths", "sum"))

In [106]:
covid_df["procent_healed"] = covid_df.sum_healed / covid_df.sum_infections
covid_df["procent_infected"] = covid_df.sum_infections / covid_df.population
covid_df["procent_death"] = covid_df.sum_deathed / covid_df.population

In [109]:
covid_df

,sum_infections,population,sum_healed,sum_deathed,procent_healed,procent_infected,procent_death
region,,,,,,,
Адыгея,168068.0,463088.0,131060.0,1367.0,0.779803,0.362929,0.002952
Алтай,199512.0,220181.0,172650.0,689.0,0.865361,0.906127,0.003129
Алтайский край,608828.0,2317153.0,567713.0,12523.0,0.932469,0.262748,0.005404
Амурская,221459.0,790044.0,208040.0,1997.0,0.939406,0.280312,0.002528
Архангельская,642644.0,1092424.0,426098.0,7967.0,0.663039,0.588273,0.007293
...,...,...,...,...,...,...,...
Челябинская,617718.0,3466369.0,464942.0,6081.0,0.752677,0.178203,0.001754
Чечня,111140.0,1478726.0,62946.0,1495.0,0.566367,0.075159,0.001011
Чувашия,299533.0,1217818.0,239282.0,5971.0,0.798850,0.245959,0.004903


В Москве суммарно число выявленных случаев заболевших больше чем численность населения на начало 2020 года - это может быть связано с повторным заражением инфекцией

In [118]:
df.shape

(75604, 22)

In [119]:
df.to_csv("dataset_1.csv", index=None)